# Random Forests Algorithm Adaptation

In [112]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [178]:
pd.set_option('display.max_columns', None)

In [179]:
full_data = pd.DataFrame(pd.read_pickle('combined_imdb_tmdb_data.picke'))
full_data.head()

,adult,backdrop_path,belongs_to_collection,budget,cast,certificates,color info,countries,director,genres,homepage,id,imdb_id,languages,mpaa,original_language,original_title,overview,plot,popularity,poster_path,production companies,production_companies,production_countries,rating,release_date,revenue,runtime,runtimes,spoken_languages,status,tagline,title,video,vote_average,vote_count,votes,writer,year
0,False,/nhhV4jSDZcBCZuOjkbPUAGa8EKG.jpg,None,0,"[Alexander Ludwig, Christopher Eccleston, Ian ...","[Argentina:13, Australia:PG, Brazil:Livre, Can...",[Color],[USA],[David L. Cunningham],"[{u'id': 12, u'name': u'Adventure'}, {u'id': 1...",,2274,tt0484562,[English],Rated PG for fantasy action and some scary images,en,The Seeker: The Dark Is Rising,A boy's life is turned upside down when he lea...,"[The film tells the story of Will Stanton, a y...",0.610288,/wmPHQxchlpuRAFG94NWMtCZF65k.jpg,"[Twentieth Century Fox Film Corporation, Walde...",[{u'name': u'Twentieth Century Fox Film Corpor...,"[{u'iso_3166_1': u'US', u'name': u'United Stat...",4.8,2007-10-05,0,94.0,[99],"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,The Seeker: The Dark Is Rising,False,4.8,88,13504.0,"[John Hodge, Susan Cooper]",2007.0
1,False,/d23XXntJrVwNENvdriJdqlW1ae6.jpg,None,3000000,"[William Holden, Alec Guinness, Jack Hawkins, ...","[Argentina:13, Australia:PG, Australia:G::(Ori...",[Color::(Technicolor)],"[UK, USA]",[David Lean],"[{u'id': 18, u'name': u'Drama'}, {u'id': 36, u...",,826,tt0050212,"[English, Japanese, Thai]",Rated PG for mild war violence (re-rating) (1991),en,The Bridge on the River Kwai,The classic story of English POWs in Burma for...,[The film deals with the situation of British ...,1.138653,/vtPR6tSHeu35rF6qTDw3Yjr9eDg.jpg,"[Columbia Pictures Corporation, Horizon Pictur...","[{u'name': u'Columbia Pictures Corporation', u...","[{u'iso_3166_1': u'GB', u'name': u'United King...",8.2,1957-10-02,33300000,161.0,[161],"[{u'iso_639_1': u'ja', u'name': u'日本語'}, {u'is...",Released,It spans a whole new world of entertainment!,The Bridge on the River Kwai,False,7.6,450,159094.0,"[Pierre Boulle, Carl Foreman, Michael Wilson]",1957.0
2,False,/2Kx2q1GOmYRjxmILEFIusMIGTUc.jpg,"{u'id': 123800, u'poster_path': u'/giaxAC74fht...",2700000,"[Ethan Hawke, Julie Delpy, Vernon Dobtcheff, L...","[Argentina:Atp, Australia:M, Austria:0, Brazil...",[Color],[USA],[Richard Linklater],"[{u'id': 18, u'name': u'Drama'}, {u'id': 10749...",http://www.warnerbros.com/?site=beforesunset#/...,80,tt0381681,"[English, French]",Rated R for language and sexual references (Ap...,en,Before Sunset,Nine years ago two strangers met by chance and...,[Early thirty-something American Jesse Wallace...,1.026352,/cIj6yWJKUjdCCO7vuZQKl0NqCQe.jpg,"[Warner Independent Pictures (WIP), Castle Roc...","[{u'name': u'Castle Rock Entertainment', u'id'...","[{u'iso_3166_1': u'US', u'name': u'United Stat...",8.0,2004-02-10,15992615,80.0,[80],"[{u'iso_639_1': u'en', u'name': u'English'}, {...",Released,What if you had a second chance with the one t...,Before Sunset,False,7.6,580,177647.0,"[Richard Linklater, Julie Delpy, Ethan Hawke, ...",2004.0
3,False,/Dmcv6XpQ9h8AoRIEhD1hayNpd8.jpg,None,0,"[Michael Bolton, Sarah Shahi, Adam Scott, Matt...","[Singapore:NC16, South Korea:18, USA:TV-MA]",[Color],[USA],"[Scott Aukerman, Akiva Schaffer]","[{u'id': 35, u'name': u'Comedy'}, {u'id': 1040...",,430780,tt6343706,[English],NaN,en,"Michael Bolton's Big, Sexy Valentine's Day Spe...",After Santa tells Michael Bolton that he needs...,[After Santa tells Michael Bolton that he need...,0.097190,/fomxGvTf5HBtaupI9Rk4UGg43OB.jpg,"[Comedy Bang! Bang! Productions, Netflix, Pass...","[{u'name': u'Netflix', u'id': 59811}]","[{u'iso_3166_1': u'US', u'name': u'United Stat...",NaN,2017-02-07,0,54.0,[USA:54],"[{u'iso_639_1': u'en', u'name': u'English'}]",Released,,"Michael Bolton's Big, Sexy Valentine's Day Spe...",False,8.0,4,NaN,"[Scott Aukerman, David Ferguson, Mike Hanford,...",2017.0
4,False,/8fYXNzpwjJIUmP6ZtxPO3zp

In [180]:
cols_to_save = ["budget", "adult", "director", "genres", "id", "original_language", "popularity", 
                "rating", "revenue", "runtime", "vote_average", "vote_count", "votes", "writer", "year"]

In [181]:
full_data_condensed = full_data[cols_to_save]

In [154]:
pd.read_pickle('train_val_test_byid.pkl')

{'test': [7840,
  363483,
  122369,
  155084,
  80295,
  2207,
  199408,
  28742,
  159014,
  2966,
  10845,
  192102,
  1271,
  53182,
  17691,
  332502,
  150,
  259954,
  15925,
  287689,
  14241,
  82657,
  185341,
  276122,
  86778,
  850,
  252171,
  100271,
  86472,
  348678,
  85770,
  398895,
  258230,
  1631,
  13361,
  34653,
  3111,
  40623,
  271805,
  12832,
  25336,
  2641,
  157293,
  2755,
  24983,
  9273,
  19989,
  44630,
  10117,
  2758,
  11540,
  57586,
  198287,
  38540,
  275619,
  11856,
  15807,
  12155,
  126889,
  679,
  19042,
  25943,
  7457,
  168672,
  126277,
  2770,
  1359,
  55377,
  11237,
  9932,
  8699,
  17889,
  24241,
  239318,
  61891,
  15157,
  12690,
  26233,
  451253,
  14169,
  26505,
  464,
  4435,
  9691,
  284470,
  134371,
  43279,
  58933,
  411677,
  10051,
  67083,
  350312,
  43790,
  10916,
  9381,
  105,
  226458,
  128150,
  10147,
  52449,
  427451,
  24248,
  13004,
  136215,
  355111,
  73456,
  14087,
  260409,
  278256,
  3

In [182]:
genres = full_data_condensed["genres"]

In [183]:
genre_ids = []
for genre_list in genres:
    genre_ids.append([genre["name"] for genre in genre_list])

In [184]:
full_data_condensed["genres"] = genre_ids

/Users/philliphuang/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [185]:
genre_names = []
for genre_sublist in genre_ids:
    genre_names += genre_sublist
genre_names = list(set(genre_names))

In [186]:
genre_names

[u'Mystery',
 u'Romance',
 u'Crime',
 u'Family',
 u'Science Fiction',
 u'Horror',
 u'Foreign',
 u'Drama',
 u'Fantasy',
 u'Western',
 u'Animation',
 u'Music',
 u'Adventure',
 u'Action',
 u'TV Movie',
 u'Comedy',
 u'Documentary',
 u'War',
 u'Thriller',
 u'History']

In [187]:
for genre in genre_names:
    full_data_condensed[genre] = [int(genre in genre_list) for genre_list in full_data_condensed["genres"]]

/Users/philliphuang/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [175]:
import numpy as np

In [189]:
np.sum(full_data_condensed["Mystery"])

1172

In [190]:
full_data_condensed.columns

Index([           u'budget',             u'adult',          u'director',
                  u'genres',                u'id', u'original_language',
              u'popularity',            u'rating',           u'revenue',
                 u'runtime',      u'vote_average',        u'vote_count',
                   u'votes',            u'writer',              u'year',
                 u'Mystery',           u'Romance',             u'Crime',
                  u'Family',   u'Science Fiction',            u'Horror',
                 u'Foreign',             u'Drama',           u'Fantasy',
                 u'Western',         u'Animation',             u'Music',
               u'Adventure',            u'Action',          u'TV Movie',
                  u'Comedy',       u'Documentary',               u'War',
                u'Thriller',           u'History'],
      dtype='object')

In [133]:
selected = pd.DataFrame({'rating': full_data['vote_average'], 
                         'adult': full_data['adult'], 
                         'budget': full_data['budget'],
                         'year': full_data['year']})

In [131]:
full_data['genres'][0]

[{u'id': 12, u'name': u'Adventure'},
 {u'id': 18, u'name': u'Drama'},
 {u'id': 14, u'name': u'Fantasy'},
 {u'id': 10751, u'name': u'Family'},
 {u'id': 53, u'name': u'Thriller'}]

In [113]:
imdb5k = pd.read_pickle('imdb5000onehot.pkl')
imdb5k.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,actor_1_name,...,i_Drama,i_Action,i_Documentary,i_Musical,i_History,i_Family,i_Fantasy,i_Game-Show,i_Sport,i_Biography
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,CCH Pounder,...,0,1,0,0,0,0,1,0,0,0
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Johnny Depp,...,0,1,0,0,0,0,1,0,0,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Christoph Waltz,...,0,1,0,0,0,0,0,0,0,0
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Tom Hardy,...,0,1,0,0,0,0,0,0,0,0
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Doug Walker,...,0,0,1,0,0,0,0,0,0,0


In [114]:
imdb5k.columns

Index([u'color', u'director_name', u'num_critic_for_reviews', u'duration',
       u'director_facebook_likes', u'actor_3_facebook_likes', u'actor_2_name',
       u'actor_1_facebook_likes', u'gross', u'actor_1_name', u'movie_title',
       u'num_voted_users', u'cast_total_facebook_likes', u'actor_3_name',
       u'facenumber_in_poster', u'plot_keywords', u'movie_imdb_link',
       u'num_user_for_reviews', u'language', u'country', u'content_rating',
       u'budget', u'title_year', u'actor_2_facebook_likes', u'imdb_score',
       u'aspect_ratio', u'movie_facebook_likes', u'i_Sci-Fi', u'i_Crime',
       u'i_Romance', u'i_Animation', u'i_Music', u'i_Comedy', u'i_War',
       u'i_Horror', u'i_Film-Noir', u'i_Adventure', u'i_News', u'i_Reality-TV',
       u'i_Thriller', u'i_Western', u'i_Mystery', u'i_Short', u'i_Drama',
       u'i_Action', u'i_Documentary', u'i_Musical', u'i_History', u'i_Family',
       u'i_Fantasy', u'i_Game-Show', u'i_Sport', u'i_Biography'],
      dtype='object')

In [115]:
# print categoricals
for col in imdb5k.columns:
    if imdb5k[col].dtype == 'object':
        print col

color
director_name
actor_2_name
actor_1_name
movie_title
actor_3_name
plot_keywords
movie_imdb_link
language
country
content_rating


In [116]:
imdb5k['plot_keywords'].head()

0               avatar|future|marine|native|paraplegic
1    goddess|marriage ceremony|marriage proposal|pi...
2                  bomb|espionage|sequel|spy|terrorist
3    deception|imprisonment|lawlessness|police offi...
4                                                  NaN
Name: plot_keywords, dtype: object

In [117]:
# one-hot encode select categoricals
# good_categories = ['color', 'country', 'content_rating']
good_categories = []
imdb_quant = pd.DataFrame()
for col in imdb5k.columns:
    if imdb5k[col].dtype != 'object':
        imdb_quant[col] = imdb5k[col]
    elif col in good_categories:
        imdb_quant = pd.concat([imdb_quant, pd.get_dummies(imdb5k[col])], axis=1)

In [118]:
# x y split
imdb5k_X = imdb_quant.ix[:,0:list(imdb_quant.columns).index('i_Sci-Fi')]
imdb5k_Y = imdb_quant.ix[:,list(imdb_quant.columns).index('i_Sci-Fi'):len(imdb_quant.columns)]

# impute
def impute(dataf):
    for col in dataf.columns:
        dataf[col].fillna(dataf[col].mean(), inplace=True)
impute(imdb5k_X)
impute(imdb5k_Y)

# train test split
x_train, x_test, y_train, y_test = train_test_split(imdb5k_X, imdb5k_Y, stratify = imdb5k_Y)

In [119]:
params = {
    'n_estimators': [100],
    'max_features': ['auto', 'log2', None, 1] + range(5, 20, 5),
    'n_jobs' : [-1]
}

CV_rf = GridSearchCV(RandomForestClassifier(), param_grid=params, cv=5, n_jobs=-1)
CV_rf.fit(x_train, y_train)
print CV_rf.best_params_
print CV_rf.best_score_
print CV_rf.score(x_test, y_test)

{'max_features': None, 'n_estimators': 100, 'n_jobs': -1}
0.486515071391
0.584456780333


In [120]:
hamming_loss(y_test, CV_rf.predict(x_test))

0.04221313975477338

In [121]:
print ['auto', 'log2', None, 1] + range(5, 30, 5)

['auto', 'log2', None, 1, 5, 10, 15, 20, 25]


In [122]:
pd.DataFrame({'feature': x_train.columns, 'importance': CV_rf.best_estimator_.feature_importances_})

,feature,importance
0,num_critic_for_reviews,0.066859
1,duration,0.107029
2,director_facebook_likes,0.056514
3,actor_3_facebook_likes,0.054535
4,actor_1_facebook_likes,0.048569
5,gross,0.074388
6,num_voted_users,0.069728
7,cast_total_facebook_likes,0.049365
8,facenumber_in_poster,0.035870
9,num_user_for_reviews,0.067889
